<a href="https://colab.research.google.com/github/aloy4646/football_video_classification/blob/main/football_video_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
import pathlib

import cv2
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, Sequential

from tensorflow.keras.layers import TimeDistributed, Conv2D, Dense, MaxPool2D, Flatten, LSTM

In [ ]:
HEIGHT = 224
WIDTH = 224

In [ ]:
class FrameGeneration:
  def __init__(self, path, n_frames, training = False):
    """ Mengembalikan set frames beserta label terkait.

      Args:
        path: Path file video.
        n_frames: Jumlah frames.
        training: Boolean untuk menentukan apakah dataset training sedang dibuat.
    """
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
          # self.path.iterdir() --> (dataset_final/train/card), (dataset_final/train/celebration), (dataset_final/train/corner)

    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    # video_paths = list(self.path.glob('*/*.avi'))
    video_paths = list(self.path.glob('*/*.mp4'))
          # video_paths --> '/dataset_final/train/substitution/*.mp4'

    classes = [p.parent.name for p in video_paths]
    return video_paths, classes

  def frames_from_video_file(self, video_path, n_frames, output_size = (HEIGHT,WIDTH)):
    """
      Membuat frames dari setiap file video yang ada untuk setiap kategori.

      Args:
        video_path: Path file video.
        n_frames: Jumlah frames yang akan dibuat per file video.
        output_size: Ukuran piksel dari frame keluaran.

      Return:
        Array NumPy dari frames dengan bentuk (n_frames, HEIGHT, WIDTH, saluran).
    """
    # Baca setiap frame video
    result = []
    src = cv2.VideoCapture(str(video_path))

    video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

    # Frame step fleksibel mengikuti panjang dari sumber video
    frame_step = max(int(video_length / (n_frames-1)), 1)

    need_length = 1 + (n_frames-1) * frame_step

    if need_length > video_length:
      start = 0
    else:
      max_start = video_length - need_length
      start = random.randint(0, max_start + 1)

    src.set(cv2.CAP_PROP_POS_FRAMES, start)

    # ret adalah boolean yang menunjukkan apakah pembacaan berhasil, frame adalah gambar itu sendiri
    ret, frame = src.read()
    result.append(self.format_frames(frame, output_size))

    for _ in range(n_frames - 1):
      for _ in range(frame_step):
        ret, frame = src.read()
      if ret and frame is not None:
        frame = self.format_frames(frame, output_size)
        result.append(frame)
      else:
        result.append(np.zeros_like(result[0]))

    src.release()
    result = np.array(result)[..., [2, 1, 0]]
          # BGR --> RGB
    return result


  def format_frames(self, frame, output_size):
      """
      Menambahkan padding dan meresize frame dari video.

      Args:
          frame: frame yang perlu diresize dan dipad.
          output_size: Ukuran piksel dari frame keluaran.

          Return:
            Frame yang diformat dengan padding sesuai ukuran keluaran.
      """
      # return resized_frame
      frame = tf.image.convert_image_dtype(frame, tf.float32)
      frame = tf.image.resize_with_pad(frame, *output_size)
      return frame

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))
                  # video_paths = ['/path/to/video1.mp4', '/path/to/video2.mp4', '/path/to/video3.mp4']
                  # classes = ['class1', 'class2', 'class3']
                  # zip --> ('/path/to/video1.mp4', 'class1'), ('/path/to/video2.mp4', 'class2'), ('/path/to/video3.mp4', 'class3')

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      # frame retrieval
      video_frames = self.frames_from_video_file(path, self.n_frames)
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

In [ ]:
dataset_dir = pathlib.Path('/content/drive/MyDrive/AnwarSukoco/dataset_final/')

n_frames = 10

# banyaknya kelas
num_classes = len(FrameGeneration(dataset_dir / 'train', n_frames, training=True).class_names)

In [ ]:
batch_size = 10

output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGeneration(dataset_dir / 'train', n_frames, training=True),
                                          output_signature = output_signature)

# Batch data
train_ds = train_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_generator(FrameGeneration(dataset_dir / 'val', n_frames),
                                        output_signature = output_signature)
val_ds = val_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(FrameGeneration(dataset_dir / 'test', n_frames),
                                         output_signature = output_signature)

test_ds = test_ds.batch(batch_size)

In [ ]:
input_shape = (batch_size, n_frames, HEIGHT, WIDTH, 3)

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu", input_shape=input_shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(TimeDistributed(MaxPool2D(pool_size=(2, 2), strides=(2, 2))))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(TimeDistributed(MaxPool2D(pool_size=(2, 2), strides=(2, 2))))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(TimeDistributed(MaxPool2D(pool_size=(2, 2), strides=(2, 2))))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(TimeDistributed(MaxPool2D(pool_size=(2, 2), strides=(2, 2))))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(TimeDistributed(MaxPool2D(pool_size=(2, 2), strides=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(1024, return_sequences=False))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 10, 224, 224, 64   1792      
                             )                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 224, 224, 64   36928     
                             )                                   
                                                                 
 time_distributed (TimeDist  (None, 10, 112, 112, 64   0         
 ributed)                    )                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 112, 112, 12   73856     
                             8)                                  
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 112, 112, 12   1

In [ ]:
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.SGD(learning_rate = 0.0001),
              metrics = ['accuracy'])

In [ ]:
model_save_path = "/content/drive/MyDrive/AnwarSukoco/TrainingHistory/model/konfigurasi_108_10_1024_150_0_0001"

checkpoint = tf.keras.callbacks.ModelCheckpoint(model_save_path+'/checkpoint_{epoch:02d}', save_freq='epoch')

csv_logger = tf.keras.callbacks.CSVLogger("/content/drive/MyDrive/AnwarSukoco/TrainingHistory/history/history108_10_1024_150_0_0001.csv", append=True)

In [ ]:
history = model.fit(x = train_ds,
                    epochs = 150,
                    validation_data = val_ds,
                    callbacks = [checkpoint, csv_logger])

Epoch 1/150
12/12 [==============================] - 238s 18s/step - loss: 1.7918 - accuracy: 0.1667 - val_loss: 1.7918 - val_accuracy: 0.1667
Epoch 2/150
12/12 [==============================] - 64s 5s/step - loss: 1.7918 - accuracy: 0.1750 - val_loss: 1.7918 - val_accuracy: 0.1667
Epoch 3/150
12/12 [==============================] - 60s 5s/step - loss: 1.7918 - accuracy: 0.1750 - val_loss: 1.7918 - val_accuracy: 0.1333
Epoch 4/150
12/12 [==============================] - 63s 5s/step - loss: 1.7918 - accuracy: 0.1750 - val_loss: 1.7918 - val_accuracy: 0.1333
Epoch 5/150
12/12 [==============================] - 60s 5s/step - loss: 1.7918 - accuracy: 0.1917 - val_loss: 1.7918 - val_accuracy: 0.1667
Epoch 6/150
12/12 [==============================] - 60s 5s/step - loss: 1.7918 - accuracy: 0.1833 - val_loss: 1.7918 - val_accuracy: 0.1667
Epoch 7/150
12/12 [==============================] - 64s 5s/step - loss: 1.7917 - accuracy: 0.1750 - val_loss: 1.7918 - val_accuracy: 0.1667
Epoch 8/150